# 1 - Simple Random Sampling

In the realm of statistics, when we're faced with a vast expanse of data - like a detailed thematic map representing diverse geographical features - it's often impractical or even impossible to examine every single point. This is where the elegance of sampling techniques comes into play.

Simple Random Sampling (SRS), as the name suggests, is the most straightforward of these techniques.  It's akin to drawing names out of a hat - every single point in our 'map' (or dataset) has an equal chance of being selected. 

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
import seaborn as sns

## 1 - Create a fake image with a predefined proportion of "change"

In this initial cell, we're essentially creating a simplified representation of a geographical area, much like a digital map. The code accomplishes this by generating a two-dimensional array (think of it as a grid or matrix) filled with random numbers.

A certain percentage of those pixels will be given the value 1 for change, and 0 for no change. The *proportion_of_change* variable will define how many points are going to be 1s.

In [ ]:
# a small helper function to create a fake image of 1s and 0s
def create_random_array(size, proportion, shuffle=True):

    # create an array of size2
    array = np.zeros(size*size)
    change_samples = int(size * size * proportion)
    
    # attribute a proportion of change
    array[:change_samples] = 1  # Assign rows to stratum 1
    
    # shuffle array
    if shuffle:
        np.random.shuffle(array)
    return array.reshape(size, size)


proportion_of_change = 0.01  # proportion of change pixels within the image (expressed as fraction)
size = 250                   # one-sided size
change_array = create_random_array(size, proportion_of_change)

# plot image
fig, ax = plt.subplots(figsize=(20, 20))
ax.imshow(change_array, interpolation='nearest', cmap='RdYlGn_r')

## 2 - Estimation of change proportion

### 2.1 - A single estimate through simple random sampling

In the first part of this cell, we calculate the true proportion of change from our array. In practise, this is not possible, as it would mean to have a perfect map or knowledge of the total population, i.e. each pixel's true value. Since satellite-based maps are hugely large and prone to some degree of error, this is impossible. 

In the second part, we sample a subset of the array. Based on the subset we calculate the proportion of change, which becomes our estimate.

If you run this cell a couple of times, you will notice that the estimate is constantly changing. This is because each time a different subset is selected, which affects the estimated proportion. 

In [ ]:
sample_size = 1000

print('Number of change samples:', np.sum(change_array))
true_proportion = np.sum(change_array)/change_array.size
print(f'{true_proportion} is our true proportion of change')

# this line randomly selects the number of samples defiend above without replacement
sampled = np.random.choice(change_array.flatten(), sample_size, replace=False)


estimated_proportion = sampled.sum()/len(sampled)
print(f'{estimated_proportion} is our estimated proportion of change')      

### 2.2 - Sampling variability and experimental proof of unbiasedness 

In this cell, we run the sampling exercise 10000 times. Each time, a different sample is taken from the full array. Therefore, each estimate is slightly different depending on how much change points are part of that sample. This is subject to the random selection and is known as **sampling variability**. Because of the random selection of a subset from your total population, a single estimate might be above or below your true value. However, if this process is repeated a lot of times, the mean of the estimates equals the true value - which means the estimator is **unbiased**.  

All estimates are stored in a DataFrame at the end.

In [ ]:
# initialize a dicitonary to be filled through out the loop
d = {}
for a in range(10000):
    
    # setting a random seed according to the loop's number of iteration
    np.random.seed(seed=a)

    # randomly sample the array created above
    sampled = np.random.choice(change_array.flatten(), sample_size, replace=False)
    
    # calculate the proportion of change based on the samples
    d[a] = [sampled.sum()/len(sampled)]

# structure data into a dataframe table
df = pd.DataFrame.from_dict(d, orient='index').reset_index()
df.columns = ['Run', 'Estimate']
df.head()

By plotting the distribution of all 10000 runs, we will notice that the mean of those runs and the true proportion (dotted red line) is exactly the same. This can be seen as an experimental proof that the applied logic is unbiased.  

In [ ]:
# initialize plot
fig, ax = plt.subplots(1)

# plot the distribution of the Estimates as a Violinplot
ax = sns.violinplot(df, y='Estimate', ax=ax, alpha=0.5)
plt.axhline(true_proportion, c='red', linestyle=':')

# Add legend for true value
handles, labels = ax.get_legend_handles_labels()
point = Line2D([0], [0], label='True value', linestyle=':', c='r')
handles.append(point) 
ax.legend(handles=handles)

plt.show()

### 2.3 - Sampling variability as a function of sample size

The more samples are used to derive an estimate of the proportion, the more likely it is to be closer to the true value, meaning the sampling variability is reduced. In the follwoing cell, we repeat the previous experiment, but adding different sample sizes. It can be seen that with a larger amount of samples the estimates are closer to the true value.

In [ ]:
sample_sizes = [1000, 3000, 5000, 7000, 9000]
iterations = 10000 # nr. of iterations per sample size

# initialize ariables to be used in the loop
d, a = {}, 0
for i in sample_sizes:
    for j in range(iterations):

        # set random seed according to the second loop's number of iteration
        np.random.seed(seed=j)

        # randomly sample the above created array with the sample size i from the first loop
        sampled = np.random.choice(change_array.flatten(), i, replace=False)
        
        # add the samplesize and calculated proportion of change to the dictionary for that specific loop
        d[a] = [i, sampled.sum()/len(sampled)]
        a += 1
        
# structure the results in a dataframe
df = pd.DataFrame.from_dict(d, orient='index')
df.columns = ['Sample Size', 'Estimate']
#display(df.head())

# plot the distribution of the Estimates as a Violinplot, grouped by sample size
fig, ax = plt.subplots(1)
ax = sns.violinplot(df, y='Estimate', x='Sample Size', hue='Sample Size', ax=ax, palette='magma', legend=False)

# Add legend for true value
handles, labels = ax.get_legend_handles_labels()
point = Line2D([0], [0], label='True value', linestyle=':', c='r')
handles.append(point) 
ax.legend(handles=handles)

plt.axhline(true_proportion, c='red', linestyle=':')
plt.show()

## 3 - Uncertainty estimation

Sampling variability belongs to the nature of sampling. While we do get a single value for our estimate, we do not know how close it is to the true value. Depending on the proportion of the phenomena among the total population and the sample size, it is possible to derive an unbiased estimate of uncertainty. Uncertainty is usually expressed as a confidence interval, and it tells us how likely our estimate can be found in a certain range around our estimate.  

The unbiasedness of the uncertainty estimate can be demonstrated. The standard deviation of a number of experimental runs equals the mean of the calculated standard error using the estimator. 
In this sense, the uncertainty expresses the range of the sampling variability. 

In [ ]:
def simple_random_sampling(array, sample_size, seed=42):

    # set random seed according to given seed
    np.random.seed(seed=seed)

    N_h = change_array.size 
    n_h = sample_size
    
    # randomly sample the given array with the given sample size
    sampled = np.random.choice(change_array.flatten(), n_h, replace=False)

    # calculate the proportion
    tau_hat = sampled.sum()/len(sampled)
    
    # calculate variance
    S = tau_hat * (1-tau_hat)

    # calculate finite correction
    finite = 1 - n_h/N_h

    # calculate the standard error
    se = np.sqrt(finite * S/(n_h - 1))

    return sample_size, tau_hat, se


d, a = {}, 0
for i in sample_sizes:
    for j in range(iterations):
        d[a] = simple_random_sampling(change_array, i, a)
        a += 1

# structure the results in a dataframe
df = pd.DataFrame.from_dict(d, orient='index')
df.columns = ['Sample Size', 'Estimate', 'SE']
display(df.head())

In [ ]:
# plot the distribution of the Estimates as a Violinplot, grouped by sample size
fig, ax = plt.subplots(1, 2, figsize=(15,8))

# first plot ( as above) with the distribution of estimates and true value
ax[0] = sns.violinplot(df, y='Estimate', x='Sample Size', hue='Sample Size', ax=ax[0], palette='magma', legend=False)
ax[0].axhline(true_proportion, c='red', linestyle=':')
ax[0].set_title('Distribution of the estimated proportion for different sampling sizes')

# Add legend for true value
handles, labels = ax[0].get_legend_handles_labels()
point = Line2D([0], [0], label='True value', linestyle=':', c='r')
handles.append(point) 
ax[0].legend(handles=handles)

# second plot with distribution of estimated SE's and calculated one (red dots)
ax[1] = sns.violinplot(df, y='SE', x='Sample Size', hue='Sample Size', ax=ax[1], palette='magma', legend=False)
ax[1] = sns.pointplot(df.groupby('Sample Size').std().reset_index(), x='Sample Size', y='Estimate', ax=ax[1], c='red', markersize=1.5, linestyle=' ')
ax[1].set_title('Distribution of the estimated SE for different sampling sizes')

# Add legend for red dots
handles, labels = ax[1].get_legend_handles_labels()
point = Line2D([0], [0], label='Calculated SE', marker='.', markersize=10, markerfacecolor='r', markeredgecolor='r', linestyle='')
handles.append(point) 
ax[1].legend(handles=handles)

plt.tight_layout()
plt.show()